In [10]:
#import package
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import matplotlib.pylab as plt
import time

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

#%matplotlib inline

In [19]:
def readData():
    data = pd.read_csv('C:/Users/yoon/Python/Stock_Prediction/Samsung20YN.csv', parse_dates = [0])
    raw_dates = data.loc[:, 'date']
    data['returns'] = pd.Series(data.loc[:, 'close'] - data.loc[:, 'open'])
    del data['date']
    raw_data = data
    return raw_data, raw_dates

def split_into_chunks(data, train, predict, step):
    train_set, label_set = [], []
    for i in range(0, len(data), step):
        try:
            temp_train = data.loc[i:i+train-1, 'close':'volume']
            temp_train = preprocessing.scale(temp_train)
            temp_label = data.loc[i+train-1+predict, 'returns']
            if temp_label > 0 :
                 temp_label = [1., 0.]
            else:
                 temp_label = [0., 1.]
        except:
            break
        train_set.append(temp_train)
        label_set.append(temp_label)
        
    return train_set, label_set

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)
    return a, b

def create_Xt_Yt(X, y, percentage=0.8):
    X_train = X[0:int(len(X) * percentage)]
    Y_train = y[0:int(len(y) * percentage)]
    X_train, Y_train = shuffle_in_unison(X_train, Y_train)
    X_test = X[int(len(X) * percentage):]
    Y_test = y[int(len(X) * percentage):]
    return X_train, X_test, Y_train, Y_test

In [20]:
timeSeriesList, dates = readData()

DATA_SIZE_ON_TRAIN = 30
TARGET_TIME = 1
STEP_SIZE = 1

X, Y = split_into_chunks(timeSeriesList, DATA_SIZE_ON_TRAIN, TARGET_TIME, STEP_SIZE)
X, Y = np.array(X), np.array(Y)
X = X.reshape(-1, DATA_SIZE_ON_TRAIN * 3)
X_train, X_test, Y_train, Y_test = create_Xt_Yt(X, Y, percentage=0.9)

In [21]:
model = Sequential()
model.add(Dense(500, input_dim = DATA_SIZE_ON_TRAIN * 3))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(250))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

print("Training...")
model.fit(X_train, 
          Y_train, 
          epochs=100, 
          batch_size = 128,
          verbose=0)

print("Evaluating...")
score = model.evaluate(X_test, Y_test, batch_size=128)
print("\nTest loss and accuracy:" + str(score))

Training...
Evaluating...
128/519 [======>.......................] - ETA: 1s
Test loss and accuracy:[0.69970629497759607, 0.49132947999847648]
